In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#reading data
movies_df = pd.read_csv('../../Dataset/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('../../Dataset/ml-latest-small/ratings.csv')

In [2]:
#making new df for calculating average rating without unnecessary features
df_for_avg_rat = ratings_df.drop(columns=['userId', 'timestamp'])
df_for_avg_rat.head()
#sorted by movieId
df_for_avg_rat.sort_values(by='movieId', inplace=True)
df_for_avg_rat
#creating average and total rating dictionaries
avr_rat = {}
tot_rat = {}
temp = 0
counter = 0
for i in range(len(df_for_avg_rat)):
    if df_for_avg_rat['movieId'].iloc[i] == df_for_avg_rat['movieId'].iloc[-1] or df_for_avg_rat['movieId'].iloc[i] == df_for_avg_rat['movieId'].iloc[i+1]:
        temp += df_for_avg_rat['rating'].iloc[i]
        counter+=1
    elif df_for_avg_rat['movieId'].iloc[i] != df_for_avg_rat['movieId'].iloc[i+1]:
        temp += df_for_avg_rat['rating'].iloc[i]
        avr_rat[df_for_avg_rat['movieId'].iloc[i]] = temp/counter
        tot_rat[df_for_avg_rat['movieId'].iloc[i]] = counter
        temp = 0
        counter = 0
    else:
        break




C:\Users\Burak\AppData\Local\Temp\ipykernel_12968\2548264972.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  avr_rat[df_for_avg_rat['movieId'].iloc[i]] = temp/counter


In [4]:
#tot_rat dictionary converted to DF
tot_rat_df = pd.DataFrame.from_dict(tot_rat, orient='index')
tot_rat_df.columns = ['totalRating']
tot_rat_df['movieId'] = tot_rat_df.index
tot_rat_df = tot_rat_df.reindex(columns=['movieId', 'totalRating'])
tot_rat_df


,movieId,totalRating
1,1,214
2,2,109
3,3,51
4,4,6
5,5,48
...,...,...
193579,193579,0
193581,193581,0
193583,193583,0
193585,193585,0


In [6]:
tot_rat_df.describe()

,movieId,totalRating
count,9723.000000,9723.000000
mean,42229.456752,9.370770
std,52171.238566,22.401955
min,1.000000,0.000000
25%,3245.000000,0.000000
50%,7299.000000,2.000000
75%,76519.500000,8.000000
max,193587.000000,328.000000


In [5]:

#avr_rat dictionary converted to DF
avr_rat_df = pd.DataFrame.from_dict(avr_rat, orient='index')
avr_rat_df.columns = ['averageRating']
avr_rat_df['movieId'] = avr_rat_df.index
avr_rat_df = avr_rat_df.reindex(columns=['movieId', 'averageRating'])
avr_rat_df


,movieId,averageRating
1,1,3.939252
2,2,3.463303
3,3,3.323529
4,4,2.750000
5,5,3.135417
...,...,...
193579,193579,inf
193581,193581,inf
193583,193583,inf
193585,193585,inf


In [ ]:
#deleting the movies that have less than 20 ratings
filtered_trd = tot_rat_df[tot_rat_df['totalRating'] >= 32]
filtered_trd.sort_values(by=['totalRating'])

#merged both DF's
ratingsWoUnusedDF = pd.merge(avr_rat_df, filtered_trd, on='movieId')
ratingsWoUnusedDF.sort_values(by=['movieId'])

CREATING USER TRAINING DATA

In [3]:
#ratings data (is going to be user data later)
ratings_df.head()
#drop timestamp column
ratings_df = ratings_df.drop(columns=['timestamp'])

In [4]:
filtered_movieids = ratingsWoUnusedDF['movieId']
filtered_movieids

0           1
1           2
2           3
3           5
4           6
        ...  
801    122882
802    122886
803    122904
804    134130
805    134853
Name: movieId, Length: 806, dtype: int64

In [5]:
ratings_df = pd.merge(ratings_df, filtered_movieids, on='movieId')
ratings_df

,userId,movieId,rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5
...,...,...,...
55654,495,1172,5.0
55655,587,1172,5.0
55656,597,1172,5.0
55657,603,1172,1.0


In [6]:
movies_df = pd.merge(movies_df, filtered_movieids, on='movieId')

movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
801,122882,Mad Max: Fury Road (2015),Action|Adventure|Sci-Fi|Thriller
802,122886,Star Wars: Episode VII - The Force Awakens (2015),Action|Adventure|Fantasy|Sci-Fi|IMAX
803,122904,Deadpool (2016),Action|Adventure|Comedy|Sci-Fi
804,134130,The Martian (2015),Adventure|Drama|Sci-Fi


In [7]:
movies_df.to_csv('./csv/movie_list.csv', index=False)

In [8]:
genres = movies_df['genres'].str.get_dummies('|')
genres

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
802,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
803,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
804,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


In [9]:
genres.drop(columns=['Film-Noir','IMAX','Musical','War','Western'], axis=1, inplace=True)
genres

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,1,1,0,0,0,0,0,0,0,0,0,0,1,1
802,1,1,0,0,0,0,0,0,1,0,0,0,1,0
803,1,1,0,0,1,0,0,0,0,0,0,0,1,0
804,0,1,0,0,0,0,0,1,0,0,0,0,1,0


In [10]:
#merging movieid's with their genres
movies_with_genres = pd.concat([movies_df,genres], axis=1)
movies_with_genres

,movieId,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,6,Heat (1995),Action|Crime|Thriller,1,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,122882,Mad Max: Fury Road (2015),Action|Adventure|Sci-Fi|Thriller,1,1,0,0,0,0,0,0,0,0,0,0,1,1
802,122886,Star Wars: Episode VII - The Force Awakens (2015),Action|Adventure|Fantasy|Sci-Fi|IMAX,1,1,0,0,0,0,0,0,1,0,0,0,1,0
803,122904,Deadpool (2016),Action|Adventure|Comedy|Sci-Fi,1,1,0,0,1,0,0,0,0,0,0,0,1,0
804,134130,The Martian (2015),Adventure|Drama|Sci-Fi,0,1,0,0,0,0,0,1,0,0,0,0,1,0


In [11]:
user_df = pd.merge(ratings_df, movies_with_genres, left_on='movieId', right_on='movieId')
user_df.drop(columns=['title','genres'], axis=1, inplace=True)
user_df.sort_values(by=['userId'])

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1,4.0,0,1,1,1,1,0,0,0,1,0,0,0,0,0
10654,1,2094,5.0,1,1,0,0,0,0,0,0,0,0,0,0,1,0
10601,1,2078,5.0,0,0,1,1,1,0,0,0,0,0,0,0,0,0
10533,1,2054,4.0,0,1,0,1,1,0,0,0,1,0,0,0,1,0
10345,1,2028,4.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52793,610,54503,5.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
40283,610,69757,3.5,0,0,0,0,1,0,0,1,0,0,0,1,0,0
40348,610,70286,4.5,0,0,0,0,0,0,0,0,0,0,1,0,1,1
16227,610,79132,4.0,1,0,0,0,0,1,0,1,0,0,1,0,1,1


In [12]:
#how many ratings each user gave as a dataframe
rating_count = user_df.pivot_table(index=['userId'], aggfunc='size')
rating_count = pd.DataFrame(rating_count)
rating_count.columns = ['rating count']
rating_count['userId'] = rating_count.index
rating_count.index.name = None
rating_count

,rating count,userId
1,152,1
2,19,2
3,18,3
4,114,4
5,37,5
...,...,...
606,352,606
607,122,607
608,460,608
609,25,609


In [13]:
"""rating_count.sort_values('rating count',inplace=True)
rating_count[:5]
users_for_test = rating_count.sort_values('rating count')[:5]
"""

"rating_count.sort_values('rating count',inplace=True)\nrating_count[:5]\nusers_for_test = rating_count.sort_values('rating count')[:5]\n"

In [13]:
"""users_for_test"""

'users_for_test'

In [14]:
"""rating_count.drop(index=[175,578,324,320,397], inplace=True)
rating_count.sort_values('userId',inplace=True)
rating_count"""

"rating_count.drop(index=[175,578,324,320,397], inplace=True)\nrating_count.sort_values('userId',inplace=True)\nrating_count"

In [15]:
"""#merge rating count with user_df
test_user_df = pd.merge(user_df, users_for_test, left_on='userId', right_on='userId')
test_user_df.sort_values(by=['userId'])"""

"#merge rating count with user_df\ntest_user_df = pd.merge(user_df, users_for_test, left_on='userId', right_on='userId')\ntest_user_df.sort_values(by=['userId'])"

In [14]:
#merge rating count with user_df
user_df = pd.merge(user_df, rating_count, left_on='userId', right_on='userId')
user_df.sort_values(by=['userId'])

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
0,1,1,4.0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,152
97,1,2115,5.0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,152
98,1,2161,5.0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,152
99,1,2174,4.0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,152
100,1,2193,4.0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32783,610,5669,4.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,379
32784,610,5679,4.0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,379
32785,610,5903,3.5,1,0,0,0,0,0,0,0,0,0,0,0,1,1,379
32776,610,2951,4.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,379


In [15]:
#getting column names for the for loop below
columnnames = user_df.columns
columnnames

Index(['userId', 'movieId', 'rating', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'rating count'],
      dtype='object')

In [16]:
#multiplying every rating by genres to find average rating each user gave for each genre
for i in range(3,17):
    user_df[columnnames[i]] = user_df[columnnames[i]] * user_df['rating']
user_df.sort_values('userId', inplace=True)
user_df


,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
0,1,1,4.0,0.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,152
97,1,2115,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,152
98,1,2161,5.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,152
99,1,2174,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,152
100,1,2193,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32783,610,5669,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,379
32784,610,5679,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,4.0,379
32785,610,5903,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,3.5,379
32776,610,2951,4.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,379


In [17]:
#replacing zero values with nan because we don't want zero value counts to affect average
user_df.replace(0, np.nan, inplace=True)
#grouping by user id so it stays the same
grouped = user_df.groupby(['userId'])

def sum_and_divide(x):
    return x.sum() / x.notnull().sum()

user_data_with_average_ratings = grouped.apply(sum_and_divide)
user_data_with_average_ratings

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
userId,,,,,,,,,,,,,,,,,,
1,1.0,1681.690789,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,NaN,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225000,152.0
2,2.0,60840.315789,3.868421,3.950000,4.166667,NaN,NaN,4.000000,3.937500,NaN,3.923077,NaN,3.000000,4.000000,4.500000,4.000000,3.857143,19.0
3,3.0,1759.666667,1.583333,3.900000,3.700000,0.500000,0.500000,0.500000,NaN,NaN,0.500000,2.000000,4.000000,NaN,0.500000,3.800000,4.666667,18.0
4,4.0,1832.842105,3.429825,3.190476,3.708333,4.000000,4.125000,3.370370,3.777778,NaN,3.314815,4.071429,4.333333,2.769231,3.218750,2.500000,3.050000,114.0
5,5.0,332.567568,3.540541,3.111111,3.250000,4.333333,4.125000,3.428571,3.600000,NaN,3.666667,4.142857,3.000000,4.000000,3.000000,2.500000,3.555556,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,606.0,8595.926136,3.676136,3.381720,3.583333,3.788462,3.620690,3.496183,3.771186,4.000000,3.857955,3.700000,3.550000,3.790323,3.772222,3.517544,3.620253,352.0
607,607.0,1696.909836,3.721311,3.703704,3.405405,3.000000,3.400000,3.300000,3.800000,NaN,4.021739,3.384615,3.937500,4.800000,3.550000,3.312500,3.973684,122.0
608,608.0,3780.973913,3.340217,3.473988,3.319549,3.109375,2.791667,3.067708,3.714286,3.333333,3.465625,3.135714,3.717949,3.815789,2.928571,3.471154,3.695489,460.0


In [18]:
#rewinding the zero values
user_data_with_average_ratings.replace(np.nan, 0, inplace=True)
#dropping movieid column because we don't need it
user_data_with_average_ratings.drop(columns='movieId', axis=1, inplace=True)
user_data_with_average_ratings

,userId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
userId,,,,,,,,,,,,,,,,,
1,1.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.000000,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225000,152.0
2,2.0,3.868421,3.950000,4.166667,0.000000,0.000000,4.000000,3.937500,0.000000,3.923077,0.000000,3.000000,4.000000,4.500000,4.000000,3.857143,19.0
3,3.0,1.583333,3.900000,3.700000,0.500000,0.500000,0.500000,0.000000,0.000000,0.500000,2.000000,4.000000,0.000000,0.500000,3.800000,4.666667,18.0
4,4.0,3.429825,3.190476,3.708333,4.000000,4.125000,3.370370,3.777778,0.000000,3.314815,4.071429,4.333333,2.769231,3.218750,2.500000,3.050000,114.0
5,5.0,3.540541,3.111111,3.250000,4.333333,4.125000,3.428571,3.600000,0.000000,3.666667,4.142857,3.000000,4.000000,3.000000,2.500000,3.555556,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,606.0,3.676136,3.381720,3.583333,3.788462,3.620690,3.496183,3.771186,4.000000,3.857955,3.700000,3.550000,3.790323,3.772222,3.517544,3.620253,352.0
607,607.0,3.721311,3.703704,3.405405,3.000000,3.400000,3.300000,3.800000,0.000000,4.021739,3.384615,3.937500,4.800000,3.550000,3.312500,3.973684,122.0
608,608.0,3.340217,3.473988,3.319549,3.109375,2.791667,3.067708,3.714286,3.333333,3.465625,3.135714,3.717949,3.815789,2.928571,3.471154,3.695489,460.0


In [19]:
#get rating count a list for repeating process
rating_count_as_list = user_data_with_average_ratings["rating count"].values.tolist()
rating_count_as_list

[152.0,
 19.0,
 18.0,
 114.0,
 37.0,
 133.0,
 114.0,
 47.0,
 17.0,
 62.0,
 51.0,
 20.0,
 22.0,
 40.0,
 105.0,
 75.0,
 92.0,
 306.0,
 292.0,
 134.0,
 214.0,
 87.0,
 63.0,
 84.0,
 19.0,
 21.0,
 87.0,
 277.0,
 51.0,
 28.0,
 41.0,
 71.0,
 81.0,
 66.0,
 21.0,
 36.0,
 20.0,
 71.0,
 84.0,
 70.0,
 152.0,
 224.0,
 81.0,
 28.0,
 279.0,
 42.0,
 71.0,
 31.0,
 17.0,
 121.0,
 172.0,
 87.0,
 29.0,
 22.0,
 44.0,
 275.0,
 98.0,
 64.0,
 17.0,
 37.0,
 192.0,
 216.0,
 382.0,
 29.0,
 203.0,
 30.0,
 634.0,
 39.0,
 40.0,
 28.0,
 37.0,
 109.0,
 55.0,
 57.0,
 99.0,
 29.0,
 43.0,
 39.0,
 97.0,
 26.0,
 149.0,
 84.0,
 147.0,
 8.0,
 54.0,
 11.0,
 46.0,
 99.0,
 18.0,
 360.0,
 9.0,
 73.0,
 56.0,
 106.0,
 62.0,
 28.0,
 47.0,
 38.0,
 96.0,
 37.0,
 52.0,
 244.0,
 147.0,
 294.0,
 27.0,
 32.0,
 34.0,
 82.0,
 29.0,
 223.0,
 65.0,
 54.0,
 26.0,
 84.0,
 81.0,
 132.0,
 17.0,
 121.0,
 17.0,
 55.0,
 188.0,
 43.0,
 38.0,
 114.0,
 38.0,
 17.0,
 21.0,
 124.0,
 28.0,
 56.0,
 244.0,
 35.0,
 33.0,
 188.0,
 77.0,
 93.0,
 7.0,
 115.0,

In [20]:
#repeat every column by userid according to the the users rating counts
repeated_user_df = user_data_with_average_ratings.loc[user_data_with_average_ratings.index.repeat(rating_count_as_list)]
repeated_user_df.rename(columns={'rating': 'average rating'}, inplace=True)
repeated_user_df


,userId,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
userId,,,,,,,,,,,,,,,,,
1,1.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225,152.0
1,1.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225,152.0
1,1.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225,152.0
1,1.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225,152.0
1,1.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225,152.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,610.0,4.067282,4.059748,4.061475,4.104167,3.980000,3.972028,4.183544,4.0,4.111111,4.080357,4.071429,4.236842,3.932692,4.107955,4.144,379.0
610,610.0,4.067282,4.059748,4.061475,4.104167,3.980000,3.972028,4.183544,4.0,4.111111,4.080357,4.071429,4.236842,3.932692,4.107955,4.144,379.0
610,610.0,4.067282,4.059748,4.061475,4.104167,3.980000,3.972028,4.183544,4.0,4.111111,4.080357,4.071429,4.236842,3.932692,4.107955,4.144,379.0


In [21]:
#reposition the columns beacuse it is easier to read
new_column_order = ['userId', 'rating count', 'average rating', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller']
repeated_user_df = repeated_user_df.reindex(columns=new_column_order)
repeated_user_df

,userId,rating count,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
userId,,,,,,,,,,,,,,,,,
1,1.0,152.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225
1,1.0,152.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225
1,1.0,152.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225
1,1.0,152.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225
1,1.0,152.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,610.0,379.0,4.067282,4.059748,4.061475,4.104167,3.980000,3.972028,4.183544,4.0,4.111111,4.080357,4.071429,4.236842,3.932692,4.107955,4.144
610,610.0,379.0,4.067282,4.059748,4.061475,4.104167,3.980000,3.972028,4.183544,4.0,4.111111,4.080357,4.071429,4.236842,3.932692,4.107955,4.144
610,610.0,379.0,4.067282,4.059748,4.061475,4.104167,3.980000,3.972028,4.183544,4.0,4.111111,4.080357,4.071429,4.236842,3.932692,4.107955,4.144


In [22]:
#creating x user training data (dropping userid, ratingcount, average rating)
x_user_train = repeated_user_df
x_user_train

,userId,rating count,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
userId,,,,,,,,,,,,,,,,,
1,1.0,152.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225
1,1.0,152.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225
1,1.0,152.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225
1,1.0,152.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225
1,1.0,152.0,4.401316,4.307692,4.444444,4.769231,4.789474,4.296875,4.343750,0.0,4.444444,4.464286,3.000000,4.400000,4.235294,4.285714,4.225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,610.0,379.0,4.067282,4.059748,4.061475,4.104167,3.980000,3.972028,4.183544,4.0,4.111111,4.080357,4.071429,4.236842,3.932692,4.107955,4.144
610,610.0,379.0,4.067282,4.059748,4.061475,4.104167,3.980000,3.972028,4.183544,4.0,4.111111,4.080357,4.071429,4.236842,3.932692,4.107955,4.144
610,610.0,379.0,4.067282,4.059748,4.061475,4.104167,3.980000,3.972028,4.183544,4.0,4.111111,4.080357,4.071429,4.236842,3.932692,4.107955,4.144


CREATING USER TEST DATA

In [23]:
#y train should contain movie ratings
y_train_user = user_df['rating']
y_train_user

0        4.0
97       5.0
98       5.0
99       4.0
100      4.0
        ... 
32783    4.0
32784    4.0
32785    3.5
32776    4.5
32611    3.5
Name: rating, Length: 55659, dtype: float64

CREATING ITEM TRAINING DATA

In [24]:
#checking ratings data. we want it to be like movieId | year | average rating | (genres)
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
801,122882,Mad Max: Fury Road (2015),Action|Adventure|Sci-Fi|Thriller
802,122886,Star Wars: Episode VII - The Force Awakens (2015),Action|Adventure|Fantasy|Sci-Fi|IMAX
803,122904,Deadpool (2016),Action|Adventure|Comedy|Sci-Fi
804,134130,The Martian (2015),Adventure|Drama|Sci-Fi


In [25]:
#subtracting the year from the movie title 
pattern = '\((\d{4})\)'
movies_df['year'] = movies_df.title.str.extract(pattern, expand=False)
movies_df.drop(columns=['title','genres'], axis=1, inplace=True)

Burada bazı filmlerin başlığında film yılı yazmadığı için NaN değer alan year değerleri vardı, filtreleme yapınca kalmadı

In [26]:
pd.isnull(movies_df['year']).sort_values()

0      False
531    False
532    False
533    False
534    False
       ...  
272    False
273    False
274    False
264    False
805    False
Name: year, Length: 806, dtype: bool

In [27]:
ratings_df.sort_values(by=['rating'])

,userId,movieId,rating
35748,22,30749,0.5
50840,219,4025,0.5
15564,551,6874,0.5
54025,256,48385,0.5
30829,416,4886,0.5
...,...,...,...
27748,243,434,5.0
27766,402,434,5.0
36696,105,92259,5.0
34240,520,34048,5.0


In [28]:
#finding how many unique movieId's
ratings_df['movieId']. nunique()

806

In [29]:
#finding average rating for each movie
grouped_movies = user_df.groupby(['movieId'])

movie_ave_rating = grouped_movies.apply(sum_and_divide)
movie_ave_rating.drop(columns=['userId', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Horror', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'rating count'], axis= 1, inplace=True)
movie_ave_rating.rename(columns={"movieId": "movieId", "rating": "average rating"}, inplace=True)
movie_ave_rating.index.name = None
movie_ave_rating

,movieId,average rating
1,1.0,3.920930
2,2.0,3.431818
3,3.0,3.259615
5,5.0,3.071429
6,6.0,3.946078
...,...,...
122882,122882.0,3.819149
122886,122886.0,3.853659
122904,122904.0,3.833333
134130,134130.0,4.000000


In [30]:
#merge average rating with movies_df
movies_df = pd.merge(movies_df, movie_ave_rating)
#x_item_train = pd.merge(movies_df[['movieId', 'year']], movie_ave_rating[['average rating']], on='movieId')
movies_df

,movieId,year,average rating
0,1,1995,3.920930
1,2,1995,3.431818
2,3,1995,3.259615
3,5,1995,3.071429
4,6,1995,3.946078
...,...,...,...
801,122882,2015,3.819149
802,122886,2015,3.853659
803,122904,2016,3.833333
804,134130,2015,4.000000


In [31]:
movies_with_genres


,movieId,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,6,Heat (1995),Action|Crime|Thriller,1,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,122882,Mad Max: Fury Road (2015),Action|Adventure|Sci-Fi|Thriller,1,1,0,0,0,0,0,0,0,0,0,0,1,1
802,122886,Star Wars: Episode VII - The Force Awakens (2015),Action|Adventure|Fantasy|Sci-Fi|IMAX,1,1,0,0,0,0,0,0,1,0,0,0,1,0
803,122904,Deadpool (2016),Action|Adventure|Comedy|Sci-Fi,1,1,0,0,1,0,0,0,0,0,0,0,1,0
804,134130,The Martian (2015),Adventure|Drama|Sci-Fi,0,1,0,0,0,0,0,1,0,0,0,0,1,0


In [32]:
#creating not repeated item dataframe
x_item_train_not_repeated = pd.merge(movies_df,movies_with_genres)
x_item_train_not_repeated.drop(columns=['title', 'genres'], axis= 1, inplace=True)
x_item_train_not_repeated

,movieId,year,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,1995,3.431818,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,1995,3.259615,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,5,1995,3.071429,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,6,1995,3.946078,1,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,122882,2015,3.819149,1,1,0,0,0,0,0,0,0,0,0,0,1,1
802,122886,2015,3.853659,1,1,0,0,0,0,0,0,1,0,0,0,1,0
803,122904,2016,3.833333,1,1,0,0,1,0,0,0,0,0,0,0,1,0
804,134130,2015,4.000000,0,1,0,0,0,0,0,1,0,0,0,0,1,0


In [33]:
user_df

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
0,1,1,4.0,NaN,4.0,4.0,4.0,4.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,152
97,1,2115,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,152
98,1,2161,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,152
99,1,2174,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,152
100,1,2193,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32783,610,5669,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,379
32784,610,5679,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,4.0,379
32785,610,5903,3.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,3.5,379
32776,610,2951,4.5,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,379


In [34]:
#we need to merge according to movieid order in user_df. using locate to get the order of the movieids
y_train_sort = user_df.loc[:, ['movieId', 'rating']]
y_train_sort

,movieId,rating
0,1,4.0
97,2115,5.0
98,2161,5.0
99,2174,4.0
100,2193,4.0
...,...,...
32783,5669,4.0
32784,5679,4.0
32785,5903,3.5
32776,2951,4.5


In [35]:
#merging
x_item_train_repeated = pd.merge(y_train_sort, x_item_train_not_repeated, on='movieId')
x_item_train_repeated.drop(columns=['rating'],inplace=True)
x_item_train_repeated

,movieId,year,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
2,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
3,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
4,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55654,1172,1989,4.161765,0,0,0,0,0,0,0,1,0,0,0,0,0,0
55655,1172,1989,4.161765,0,0,0,0,0,0,0,1,0,0,0,0,0,0
55656,1172,1989,4.161765,0,0,0,0,0,0,0,1,0,0,0,0,0,0
55657,1172,1989,4.161765,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [36]:
#dropping moiveid because it's not gonna be needed
x_train_item = x_item_train_repeated
x_train_item

,movieId,year,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
2,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
3,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
4,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55654,1172,1989,4.161765,0,0,0,0,0,0,0,1,0,0,0,0,0,0
55655,1172,1989,4.161765,0,0,0,0,0,0,0,1,0,0,0,0,0,0
55656,1172,1989,4.161765,0,0,0,0,0,0,0,1,0,0,0,0,0,0
55657,1172,1989,4.161765,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [39]:
"""x_train_item.drop(columns=['year'],inplace=True)
x_train_item"""

"x_train_item.drop(columns=['year'],inplace=True)\nx_train_item"

In [37]:
x_train_item.to_csv('./csv/x_train_item.csv', index=False, header=False)
x_user_train.to_csv('./csv/x_train_user.csv', index=False, header=False)
y_train_user.to_csv('./csv/y_train.csv', index=False, header=False)

In [38]:
x_item_train_not_repeated.to_csv('./csv/item_vecs.csv', index=False, header=False)

In [39]:
user_movie_rating = user_df[['userId','movieId','rating']]
user_movie_rating

,userId,movieId,rating
0,1,1,4.0
97,1,2115,5.0
98,1,2161,5.0
99,1,2174,4.0
100,1,2193,4.0
...,...,...,...
32783,610,5669,4.0
32784,610,5679,4.0
32785,610,5903,3.5
32776,610,2951,4.5


In [40]:
inner_dict_name = 'movies'
def create_inner_dict(group):
    return {inner_dict_name: dict(zip(group['movieId'], group['rating']))}

umr_dict = user_movie_rating.groupby('userId').apply(create_inner_dict).to_dict()

In [41]:
umr_dict

{1: {'movies': {1: 4.0,
   2115: 5.0,
   2161: 5.0,
   2174: 4.0,
   2193: 4.0,
   2268: 4.0,
   2273: 4.0,
   2291: 5.0,
   2329: 5.0,
   2353: 5.0,
   2395: 5.0,
   2406: 4.0,
   2427: 5.0,
   2470: 5.0,
   2502: 5.0,
   2529: 5.0,
   2105: 4.0,
   2542: 5.0,
   2094: 5.0,
   2054: 4.0,
   1552: 4.0,
   1573: 5.0,
   1580: 3.0,
   1617: 5.0,
   1625: 5.0,
   1676: 3.0,
   1732: 5.0,
   1777: 4.0,
   1954: 5.0,
   1967: 4.0,
   2000: 4.0,
   2005: 5.0,
   2012: 4.0,
   2018: 5.0,
   2028: 4.0,
   2078: 5.0,
   2571: 5.0,
   2580: 5.0,
   2617: 2.0,
   3052: 5.0,
   3147: 5.0,
   3176: 1.0,
   3247: 3.0,
   3253: 5.0,
   3448: 5.0,
   3489: 4.0,
   3527: 4.0,
   3578: 5.0,
   3617: 4.0,
   3671: 5.0,
   3702: 5.0,
   3703: 5.0,
   3793: 5.0,
   5060: 5.0,
   3034: 5.0,
   3033: 5.0,
   2997: 4.0,
   2987: 5.0,
   2628: 4.0,
   2640: 4.0,
   2641: 5.0,
   2657: 3.0,
   2692: 5.0,
   2700: 5.0,
   2716: 5.0,
   1517: 5.0,
   2761: 5.0,
   2858: 5.0,
   2916: 4.0,
   2947: 5.0,
   2948: 5

In [42]:
with open('umr_dict.pickle', 'wb') as f:
    # Dump the dictionary into the file using pickle
    pickle.dump(umr_dict, f)